In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn import metrics
import matplotlib.pyplot as plt

# Load the dataset
dataset = pd.read_csv('MachineLearning/cancer-patient-data-sets.csv')
dataset = dataset.iloc[:, 2:]

# Separate features (x) and target variable (y)
x = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

# Map class labels to binary values
class_mapping = {"Low": 0, "Medium": 1, "High": 1}
y = y.map(class_mapping)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=42, stratify=y)

# Build SVM model
svm_model = SVC(probability=True)  # Enable probability estimation for ROC curve later
svm_model.fit(x_train, y_train)

# Predictions on the test set
y_pred = svm_model.predict(x_test)

# Evaluate model performance
accuracy = metrics.accuracy_score(y_test, y_pred)
f1_score = metrics.f1_score(y_test, y_pred)

print("\nWithout Tuned Model Performance:")
print("Accuracy:", accuracy)
print("F1 Score:", f1_score)

# Hyperparameter tuning using GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf','poly'], 'gamma': ['scale', 'auto']}
grid_search = GridSearchCV(SVC(probability=True), param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# Best hyperparameters
best_params = grid_search.best_params_
print("\nBest Hyperparameters:", best_params)

# Refit the model with the best hyperparameters
best_svm_model = grid_search.best_estimator_

# Predictions on the test set with the tuned model
y_pred_tuned = best_svm_model.predict(x_test)

# Evaluate tuned model performance
accuracy_tuned = metrics.accuracy_score(y_test, y_pred_tuned)
f1_score_tuned = metrics.f1_score(y_test, y_pred_tuned)

print("\nTuned Model Performance:")
print("Accuracy:", accuracy_tuned)
print("F1 Score:", f1_score_tuned)



# Manual hyperparameter tuning
C_values = [0.1, 1, 10, 100]
kernel_values = ['linear', 'rbf','poly']
gamma_values = ['scale', 'auto',0.1,0,1,10,100]

best_accuracy = 0
best_params_manual = None

# Iterate through hyperparameter combinations
for C in C_values:
    for kernel in kernel_values:
        for gamma in gamma_values:
            # Create and train SVM model
            svm_model_manual = SVC(C=C, kernel=kernel, gamma=gamma, probability=True)

            # Perform cross-validation and calculate mean accuracy
            accuracies = cross_val_score(svm_model_manual, x_train, y_train, cv=5, scoring='accuracy')
            mean_accuracy = accuracies.mean()

            # Update best parameters if the current combination has higher accuracy
            if mean_accuracy > best_accuracy:
                best_accuracy = mean_accuracy
                best_params_manual = {'C': C,'gamma': gamma, 'kernel': kernel}

# Train the model with the best hyperparameters
best_svm_model_manual = SVC(
    C=best_params_manual['C'], gamma=best_params_manual['gamma'], kernel=best_params_manual['kernel'], probability=True
)
best_svm_model_manual.fit(x_train, y_train)

# Evaluate the model on the test set
y_pred_tuned_manual = best_svm_model_manual.predict(x_test)

# Calculate accuracy and F1 score manually
accuracy_tuned_manual = sum(y_pred_tuned_manual == y_test) / len(y_test)
precision_tuned_manual = sum((y_test == 1) & (y_pred_tuned_manual == 1)) / sum(y_pred_tuned_manual == 1) if sum(y_pred_tuned_manual == 1) != 0 else 0
recall_tuned_manual = sum((y_test == 1) & (y_pred_tuned_manual == 1)) / sum(y_test == 1) if sum(y_test == 1) != 0 else 0
f1_score_tuned_manual = 2 * (precision_tuned_manual * recall_tuned_manual) / (precision_tuned_manual + recall_tuned_manual) if (precision_tuned_manual + recall_tuned_manual) != 0 else 0

# Display results
print("\nBest Hyperparameters (Manually Tuned):", best_params_manual)
print("Accuracy (Manually Calculated):", accuracy_tuned_manual)
print("F1 Score (Manually Calculated):", f1_score_tuned_manual)


Without Tuned Model Performance:
Accuracy: 0.9833333333333333
F1 Score: 0.988179669030733

Best Hyperparameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}

Tuned Model Performance:
Accuracy: 1.0
F1 Score: 1.0

Best Hyperparameters (Manually Tuned): {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy (Manually Calculated): 1.0
F1 Score (Manually Calculated): 1.0
